In [1]:
# Cell 1: Create folder structure
import os

dirs = [
    "project",
    "project/agents",
    "project/tools",
    "project/memory",
    "project/core",
]

for d in dirs:
    os.makedirs(d, exist_ok=True)


In [2]:
%%writefile project/__init__.py
"""
Project package initializer.
"""


Writing project/__init__.py


In [3]:
%%writefile project/agents/__init__.py
"""
Agents package initializer.
"""


Writing project/agents/__init__.py


In [4]:
%%writefile project/tools/__init__.py
"""
Tools package initializer.
"""


Writing project/tools/__init__.py


In [5]:
%%writefile project/memory/__init__.py
"""
Memory package initializer.
"""


Writing project/memory/__init__.py


In [6]:
%%writefile project/core/__init__.py
"""
Core utilities package initializer.
"""


Writing project/core/__init__.py


In [7]:
%%writefile project/tools/tools.py
from typing import List, Dict
import re
from collections import Counter


class TranscriptFetcher:
    """
    Dummy YouTube transcript fetcher.

    In a real deployment, replace this with actual YouTube API / transcript fetching logic.
    """

    def fetch(self, url_or_query: str) -> str:
        # For demo purposes, we simulate a transcript.
        base_text = (
            "Welcome to this YouTube video. In this session, we discuss how to convert videos into blog articles. "
            "We cover the main ideas, break down the content into sections, and show how AI can automate this process. "
            "By the end, you will understand how to design a YouTube to blog converter using multi-agent systems."
        )
        simulated = f"Simulated transcript for: {url_or_query}\n\n{base_text}"
        return simulated


class SimpleSummarizer:
    """
    Very simple extractive summarizer based on sentence splitting.
    """

    def summarize(self, text: str, max_sentences: int = 3) -> str:
        if not text:
            return ""
        # Split on '.', '!', '?'
        sentences = re.split(r'(?<=[.!?])\s+', text.strip())
        sentences = [s.strip() for s in sentences if s.strip()]
        return " ".join(sentences[:max_sentences])


class SEOKeywordGenerator:
    """
    Naive keyword extractor based on word frequency.
    """

    def generate(self, text: str, max_keywords: int = 8) -> List[str]:
        if not text:
            return []
        # Lowercase and keep simple words
        tokens = re.findall(r"[a-zA-Z]{4,}", text.lower())
        stopwords = {
            "this", "that", "with", "from", "your", "will", "into", "about",
            "have", "there", "their", "which", "such", "also", "been", "they",
            "them", "then", "than", "when", "where", "what", "would", "could",
            "should", "video", "youtube", "using"
        }
        tokens = [t for t in tokens if t not in stopwords]
        freq = Counter(tokens)
        most_common = [w for w, _ in freq.most_common(max_keywords)]
        return most_common


def estimate_reading_time(text: str, words_per_minute: int = 200) -> float:
    """
    Rough estimate of reading time in minutes.
    """
    words = re.findall(r"\w+", text)
    if not words:
        return 0.0
    return max(0.1, len(words) / float(words_per_minute))


Writing project/tools/tools.py


In [8]:
%%writefile project/core/context_engineering.py
from typing import List


def chunk_text(text: str, max_chars: int = 1000) -> List[str]:
    """
    Naive text chunker that splits on paragraphs and respects a max character budget.
    """
    if not text:
        return []

    paragraphs = [p.strip() for p in text.split("\n") if p.strip()]
    chunks: List[str] = []
    current = ""

    for para in paragraphs:
        if not current:
            current = para
        elif len(current) + len(para) + 1 <= max_chars:
            current = current + "\n" + para
        else:
            chunks.append(current)
            current = para

    if current:
        chunks.append(current)

    return chunks


def compact_context(chunks: List[str], max_chunks: int = 3) -> str:
    """
    Simple context compaction: keep first N chunks.
    """
    return "\n\n".join(chunks[:max_chunks])


Writing project/core/context_engineering.py


In [9]:
%%writefile project/core/observability.py
import logging
from typing import Optional

_LOGGER_INITIALIZED = False


def _init_logging() -> None:
    global _LOGGER_INITIALIZED
    if not _LOGGER_INITIALIZED:
        logging.basicConfig(
            level=logging.INFO,
            format="%(asctime)s [%(levelname)s] %(name)s: %(message)s",
        )
        _LOGGER_INITIALIZED = True


def get_logger(name: Optional[str] = None) -> logging.Logger:
    """
    Returns a module-level logger configured with a simple formatter.
    """
    _init_logging()
    return logging.getLogger(name or "project")


Writing project/core/observability.py


In [10]:
%%writefile project/core/a2a_protocol.py
from dataclasses import dataclass, field
from typing import Any, Dict, Optional


@dataclass
class AgentMessage:
    """
    Message format for agent-to-agent communication.
    """
    sender: str
    receiver: str
    task: str
    payload: Dict[str, Any] = field(default_factory=dict)
    meta: Optional[Dict[str, Any]] = None


class A2AProtocol:
    """
    Simple helper to build messages between agents.
    """

    def build_message(
        self,
        sender: str,
        receiver: str,
        task: str,
        payload: Dict[str, Any],
        meta: Optional[Dict[str, Any]] = None,
    ) -> AgentMessage:
        return AgentMessage(
            sender=sender,
            receiver=receiver,
            task=task,
            payload=payload,
            meta=meta,
        )


Writing project/core/a2a_protocol.py


In [11]:
%%writefile project/memory/session_memory.py
from typing import Any, Dict


class SessionMemory:
    """
    Simple in-memory key-value store for agent sessions.
    """

    def __init__(self) -> None:
        self._store: Dict[str, Any] = {}

    def set(self, key: str, value: Any) -> None:
        self._store[key] = value

    def get(self, key: str, default: Any = None) -> Any:
        return self._store.get(key, default)

    def to_dict(self) -> Dict[str, Any]:
        return dict(self._store)


Writing project/memory/session_memory.py


In [12]:
%%writefile project/agents/planner.py
from typing import Dict, Any

from project.tools.tools import TranscriptFetcher
from project.core.context_engineering import chunk_text
from project.core.observability import get_logger
from project.memory.session_memory import SessionMemory


class Planner:
    """
    Planner agent for the YouTube → Blog Article Converter.

    Responsibilities:
    - Fetch a transcript (simulated here)
    - Break it into sections
    - Build a high-level plan for the worker
    """

    def __init__(self) -> None:
        self.logger = get_logger("Planner")
        self.transcript_fetcher = TranscriptFetcher()

    def create_plan(self, user_input: str, memory: SessionMemory) -> Dict[str, Any]:
        self.logger.info("Creating plan for user input.")
        style_prefs = memory.get(
            "style_preferences",
            {"tone": "simple", "length": "medium"},
        )

        transcript = self.transcript_fetcher.fetch(user_input)
        sections = chunk_text(transcript, max_chars=800)

        plan: Dict[str, Any] = {
            "task": "youtube_to_blog",
            "original_input": user_input,
            "transcript": transcript,
            "sections": sections,
            "style": style_prefs,
        }

        memory.set("last_plan", plan)
        self.logger.info("Plan created with %d sections.", len(sections))
        return plan


Writing project/agents/planner.py


In [13]:
%%writefile project/agents/worker.py
from typing import Dict, Any, List

from project.tools.tools import SimpleSummarizer, SEOKeywordGenerator, estimate_reading_time
from project.core.observability import get_logger
from project.memory.session_memory import SessionMemory


class Worker:
    """
    Worker agent that turns a transcript plan into a blog-style article.
    """

    def __init__(self) -> None:
        self.logger = get_logger("Worker")
        self.summarizer = SimpleSummarizer()
        self.keyword_gen = SEOKeywordGenerator()

    def generate_blog(self, plan: Dict[str, Any], memory: SessionMemory) -> Dict[str, Any]:
        self.logger.info("Generating blog content from plan.")
        transcript: str = plan.get("transcript", "")
        sections: List[str] = plan.get("sections", [])
        style = plan.get("style", {})

        overall_summary = (
            self.summarizer.summarize(transcript, max_sentences=3)
            if transcript
            else ""
        )
        keywords = self.keyword_gen.generate(transcript)
        est_read_time = estimate_reading_time(transcript)

        title = "Blog Article based on YouTube Video"
        if keywords:
            title = f"{keywords[0].capitalize()} – A Blog Based on a YouTube Video"

        body_parts: List[str] = []
        for idx, sec in enumerate(sections, start=1):
            sec_summary = self.summarizer.summarize(sec, max_sentences=2)
            body_parts.append(f"## Section {idx}\n\n{sec_summary}\n")

        header_meta = [
            f"# {title}",
            "",
            f"**Estimated reading time:** {est_read_time:.1f} minutes",
            "",
        ]

        if overall_summary:
            header_meta.append(f"**Summary:** {overall_summary}\n")

        if keywords:
            header_meta.append(f"**SEO Keywords:** {', '.join(keywords)}\n")

        article_body = "\n".join(header_meta) + "\n".join(body_parts)

        draft: Dict[str, Any] = {
            "title": title,
            "summary": overall_summary,
            "keywords": keywords,
            "body": article_body,
            "style": style,
        }

        memory.set("last_draft", draft)
        self.logger.info("Draft generated with length %d characters.", len(article_body))
        return draft


Writing project/agents/worker.py


In [14]:
%%writefile project/agents/evaluator.py
from typing import Dict, Any

from project.core.observability import get_logger
from project.memory.session_memory import SessionMemory


class Evaluator:
    """
    Evaluator agent that scores and returns the final article.
    """

    def __init__(self) -> None:
        self.logger = get_logger("Evaluator")

    def evaluate(self, draft: Dict[str, Any], plan: Dict[str, Any], memory: SessionMemory) -> Dict[str, Any]:
        self.logger.info("Evaluating draft.")
        body = draft.get("body", "").strip()

        if not body:
            score = 0.0
            feedback = "No article content generated."
            final_article = "No article generated."
        else:
            score = 1.0
            feedback = "For this demo, the generated article looks acceptable."
            final_article = body

        result: Dict[str, Any] = {
            "score": score,
            "feedback": feedback,
            "final_article": final_article,
        }

        memory.set("last_evaluation", result)
        self.logger.info("Evaluation complete with score %.2f.", score)
        return result


Writing project/agents/evaluator.py


In [15]:
%%writefile project/main_agent.py
from typing import Dict, Any

from project.agents.planner import Planner
from project.agents.worker import Worker
from project.agents.evaluator import Evaluator
from project.memory.session_memory import SessionMemory
from project.core.observability import get_logger
from project.core.a2a_protocol import A2AProtocol, AgentMessage


class MainAgent:
    """
    Orchestrates Planner → Worker → Evaluator for the YouTube → Blog Article Converter.
    """

    def __init__(self) -> None:
        self.logger = get_logger("MainAgent")
        self.memory = SessionMemory()
        self.planner = Planner()
        self.worker = Worker()
        self.evaluator = Evaluator()
        self.protocol = A2AProtocol()

    def handle_message(self, user_input: str) -> Dict[str, Any]:
        self.logger.info("Handling user input through MainAgent.")

        plan_msg: AgentMessage = self.protocol.build_message(
            sender="user",
            receiver="planner",
            task="plan_youtube_to_blog",
            payload={"user_input": user_input},
        )

        plan = self.planner.create_plan(
            user_input=plan_msg.payload["user_input"],
            memory=self.memory,
        )

        worker_msg: AgentMessage = self.protocol.build_message(
            sender="planner",
            receiver="worker",
            task="generate_blog",
            payload={"plan": plan},
        )

        draft = self.worker.generate_blog(
            plan=worker_msg.payload["plan"],
            memory=self.memory,
        )

        evaluator_msg: AgentMessage = self.protocol.build_message(
            sender="worker",
            receiver="evaluator",
            task="evaluate_blog",
            payload={"draft": draft, "plan": plan},
        )

        evaluation = self.evaluator.evaluate(
            draft=evaluator_msg.payload["draft"],
            plan=evaluator_msg.payload["plan"],
            memory=self.memory,
        )

        response_text = evaluation.get("final_article", "")

        result: Dict[str, Any] = {
            "response": response_text,
            "plan": plan,
            "draft": draft,
            "evaluation": evaluation,
        }

        self.logger.info("MainAgent finished processing.")
        return result


def run_agent(user_input: str):
    agent = MainAgent()
    result = agent.handle_message(user_input)
    return result["response"]


Writing project/main_agent.py


In [16]:
%%writefile project/app.py
from typing import Optional

from project.main_agent import run_agent


def demo_app(user_input: Optional[str] = None) -> str:
    """
    Minimal entrypoint for using the agent in an app context.
    """
    if user_input is None:
        user_input = "https://www.youtube.com/watch?v=dummy_demo"
    return run_agent(user_input)


if __name__ == "__main__":
    print(demo_app())


Writing project/app.py


In [17]:
%%writefile project/run_demo.py
import sys
import os

sys.path.insert(0, os.path.abspath(os.path.join(os.path.dirname(__file__), "..")))

from project.main_agent import run_agent


if __name__ == "__main__":
    print(run_agent("Hello! This is a demo."))


Writing project/run_demo.py


In [18]:
%%writefile project/requirements.txt
rich
youtube-transcript-api
fastapi
uvicorn
streamlit


Writing project/requirements.txt


In [19]:
# Cell: Test the setup
from project.main_agent import run_agent

print(run_agent("Hello!"))


# Blog – A Blog Based on a YouTube Video

**Estimated reading time:** 0.3 minutes

**Summary:** Simulated transcript for: Hello! Welcome to this YouTube video. In this session, we discuss how to convert videos into blog articles.

**SEO Keywords:** blog, simulated, transcript, hello, welcome, session, discuss, convert
## Section 1

Simulated transcript for: Hello! Welcome to this YouTube video.


In [20]:
# Cell: Zip the project
!zip -r project.zip project


  adding: project/ (stored 0%)
  adding: project/app.py (deflated 39%)
  adding: project/memory/ (stored 0%)
  adding: project/memory/session_memory.py (deflated 51%)
  adding: project/memory/__init__.py (stored 0%)
  adding: project/memory/__pycache__/ (stored 0%)
  adding: project/memory/__pycache__/session_memory.cpython-312.pyc (deflated 43%)
  adding: project/memory/__pycache__/__init__.cpython-312.pyc (deflated 17%)
  adding: project/requirements.txt (deflated 2%)
  adding: project/__init__.py (stored 0%)
  adding: project/__pycache__/ (stored 0%)
  adding: project/__pycache__/main_agent.cpython-312.pyc (deflated 46%)
  adding: project/__pycache__/__init__.cpython-312.pyc (deflated 19%)
  adding: project/run_demo.py (deflated 32%)
  adding: project/tools/ (stored 0%)
  adding: project/tools/tools.py (deflated 57%)
  adding: project/tools/__init__.py (stored 0%)
  adding: project/tools/__pycache__/ (stored 0%)
  adding: project/tools/__pycache__/tools.cpython-312.pyc (deflated 42%